In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
from scripts.data_builder import HydroForecastData
from torch.utils.data import DataLoader
import torch
import glob

In [6]:
geo_folder = '../geo_data/great_db'

forecast_data = HydroForecastData(
    nc_files=glob.glob(f'{geo_folder}/nc_concat/*.nc')[:5],
    gauges=['5746', '3159', '8376'],
    predictors=['t_max_e5l', 't_min_e5l', 'prcp_e5l'],
    target=['lvl_sm'],
    hydroatlas_path='../geo_data/static_attributes/geo_vector.csv',
    hydroatlas=['for_pc_sse', 'crp_pc_sse', 'inu_pc_ult',
                'ire_pc_sse', 'lka_pc_use', 'prm_pc_sse',
                'pst_pc_sse', 'cly_pc_sav', 'slt_pc_sav',
                'snd_pc_sav', 'kar_pc_sse', 'urb_pc_sse',
                'gwt_cm_sav', 'lkv_mc_usu', 'rev_mc_usu',
                'sgr_dk_sav', 'slp_dg_sav', 'ws_area',
                'ele_mt_sav'],
    h_bs_file='../ais_parsers/data/height_gauge.csv',
    future_interval=7, past_interval=365,
    train_start='01/01/2008', train_end='12/31/2015',
    val_start='01/01/2016', val_end='12/31/2018',
    test_start='01/01/2019', test_end='12/31/2020')
# store static data for further concat in decoder
static_attributes = forecast_data.static_attributes
# Here we are defining properties for our model

# Training batch size
BATCH_SIZE = 128
# so called torch Datasets
train_ds, val_ds, test_ds = forecast_data.train_val_test()
# Split the data according to our split ratio and load each subset into a
# separate DataLoader object
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, drop_last=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, drop_last=True)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, drop_last=True)

dataloaders = {x: dl for x, dl
               in zip(['train', 'val', 'test'],
                      [train_loader, val_loader, test_loader])}
datasizes = {x: len(dl) for x, dl in dataloaders.items()}
# Observe that all parameters are being optimized
# Device selection (CPU | GPU)
USE_CUDA = torch.cuda.is_available()
device = 'cuda' if USE_CUDA else 'cpu'
print(f"We'll utlize {device} in further calculations")

We'll utlize cpu in further calculations
